In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI SDK: AutoML tabular forecasting model for batch prediction

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/automl/sdk_automl_tabular_forecasting_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fautoml%2Fsdk_automl_tabular_forecasting_batch.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/automl/sdk_automl_tabular_forecasting_batch.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/automl/sdk_automl_tabular_forecasting_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>


## Overview


This tutorial demonstrates how to use the Vertex AI SDK to create tabular forecasting models and generate batch prediction using a Google Cloud [AutoML](https://cloud.google.com/vertex-ai/docs/start/automl-users) model.

Learn more about [Forecasting with AutoML](https://cloud.google.com/vertex-ai/docs/tabular-data/forecasting/overview).

### Objective

In this tutorial, you learn how to create an AutoML tabular forecasting model from a Python script, and then generate batch prediction using the Vertex AI SDK. You can alternatively create and deploy models using the `gcloud` command-line tool or online using the Cloud Console.

This tutorial uses the following Google Cloud ML services:

- AutoML Training
- Vertex AI batch prediction
- Vertex AI model resource

The steps performed include:

- Create a Vertex AI dataset resource.
- Train an AutoML tabular forecasting model resource.
- Obtain the evaluation metrics for the model resource.
- Make a batch prediction.

### Dataset

The dataset used for this tutorial is a time series dataset containing samples drawn from the Iowa Liquor Retail Sales dataset. Data is made available by the Iowa Department of Commerce. It's provided under the Creative Commons Zero v1.0 Universal license. For more details, see: https://console.cloud.google.com/marketplace/product/iowa-department-of-commerce/iowa-liquor-sales. This dataset doesn't require any feature engineering. The version of the dataset used in this tutorial is stored in BigQuery.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Get started

### Install Vertex AI SDK for Python and other required packages


In [1]:
! pip3 install --upgrade --quiet google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 36.9 MB/s eta 0:00:00


### Restart runtime (Colab only)

To use the newly installed packages, you must restart the runtime on Google Colab.

In [2]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

Authenticate your environment on Google Colab.


In [1]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information

To get started using Vertex AI, you must have an existing Google Cloud project. Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [2]:
PROJECT_ID = "asn3-454801"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

Updated property [core/project].


### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

In [3]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

**If your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [4]:
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

Creating gs://your-bucket-name-asn3-454801-unique/...


### Import libraries and define constants

In [5]:
import urllib

from google.cloud import aiplatform, bigquery

## Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [6]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

## Tutorial

Now you're ready to begin creating your own AutoML tabular forecasting model.

#### Location of BigQuery training data.

Now set the variable `TRAINING_DATASET_BQ_PATH` to the location of the BigQuery table.

In [7]:
TRAINING_DATASET_BQ_PATH = (
    "bq://bigquery-public-data:iowa_liquor_sales_forecasting.2020_sales_train"
)

### Create the Dataset

Next, create the dataset resource by using the `create` method of the `TimeSeriesDataset` class, which takes the following parameters:

- `display_name`: The human readable name for the dataset resource.
- `gcs_source`: A list of one or more dataset index files to import the data items into the dataset resource.
- `bq_source`: Alternatively, import data items from a BigQuery table into the dataset resource.

This operation may take several minutes.

In [8]:
dataset = aiplatform.TimeSeriesDataset.create(
    display_name="iowa_liquor_sales_train",
    bq_source=[TRAINING_DATASET_BQ_PATH],
)

time_column = "date"
time_series_identifier_column = "store_name"
target_column = "sale_dollars"

print(dataset.resource_name)

projects/625784296901/locations/us-central1/datasets/275258887723024384


In [9]:
COLUMN_SPECS = {
    time_column: "timestamp",
    target_column: "numeric",
    "city": "categorical",
    "zip_code": "categorical",
    "county": "categorical",
}

### Create and run training job

To train an AutoML model, you perform two steps:
1) Create a training job.
2) Specify your training parameters and run the job.

#### Create training job

An AutoML training job is created using the `AutoMLForecastingTrainingJob` class, with the following parameters:

- `display_name`: The human readable name for the training job resource.
- `column_transformations`: (Optional): Transformations to apply to the input columns
- `optimization_objective`: The optimization objective to minimize or maximize.
    - `minimize-rmse`
    - `minimize-mae`
    - `minimize-rmsle`

The instantiated object is the job for the training pipeline.

In [10]:
MODEL_DISPLAY_NAME = "iowa-liquor-sales-forecast-model"

training_job = aiplatform.AutoMLForecastingTrainingJob(
    display_name=MODEL_DISPLAY_NAME,
    optimization_objective="minimize-rmse",
    column_specs=COLUMN_SPECS,
)

#### Run the training pipeline

Next, start the training job by invoking the `run` method, with the following parameters:

- `dataset`: The dataset resource to train the model.
- `target_column`: The column in the dataset that contains the values the model is trying to forecast.
- `time_column`: Time-series column for the forecast model.
- `time_series_identifier_column`: ID column for the time-series column.
- `available_at_forecast_columns`: List of columns that are available at the time of forecasting.
- `unavailable_at_forecast_columns`: List of columns that aren't available at the time of forecasting.
- `time_series_attribute_columns`: Columns that contain attributes or metadata related to the time series data, such as "city," "zip_code," and "county" in this example. These attributes can help the model understand the context of the time series.
- `forecast_horizon`: It determines how far into the future you want to predict, representing the number of time steps ahead for which the model generates predictions.
- `context_window`: The number of historical time steps the model uses as context for making predictions. A context window of 30 means the model uses data from the past 30 time steps to forecast future values.
- `data_granularity_unit`: The unit of time used for granularity in the data, such as "day" or "hour." This specifies the time interval between data points.
- `data_granularity_count`: The count of the granularity unit. For example, a data_granularity_count of 1 with a `data_granularity_unit` of "day" means each data point represents one day.
- `weight_column`: This parameter lets you assign different weights to different data points in your training set.
- `budget_milli_node_hours`: Maximum training time specified in unit of millihours (1000 = hour).
- `model_display_name`: The human readable name for the trained model.
- `predefined_split_column_name`: The name of a column used to specify predefined splits for training and evaluation. If not used, it’s set to `None`.

The `run` method when completed returns the model resource.

The execution of the training pipeline may take up to one hour.

In [11]:
model = training_job.run(
    dataset=dataset,
    target_column=target_column,
    time_column=time_column,
    time_series_identifier_column=time_series_identifier_column,
    available_at_forecast_columns=[time_column],
    unavailable_at_forecast_columns=[target_column],
    time_series_attribute_columns=["city", "zip_code", "county"],
    forecast_horizon=30,
    context_window=30,
    data_granularity_unit="day",
    data_granularity_count=1,
    weight_column=None,
    budget_milli_node_hours=1000,
    model_display_name=MODEL_DISPLAY_NAME,
    predefined_split_column_name=None,
)

## Review model evaluation scores

Once your model training is complete, you can examine the evaluation scores to assess the model performance

In [12]:
model_evaluations = model.list_model_evaluations()

for model_evaluation in model_evaluations:
    print(model_evaluation.to_dict())

{'name': 'projects/625784296901/locations/us-central1/models/7896108670012882944@1/evaluations/8777092548807697602', 'metricsSchemaUri': 'gs://google-cloud-aiplatform/schema/modelevaluation/forecasting_metrics_1.0.0.yaml', 'metrics': {'rootMeanSquaredError': 9114.087, 'meanAbsoluteError': 3984.9714, 'weightedAbsolutePercentageError': 47.1202, 'meanAbsolutePercentageError': 383.64523, 'rSquared': 0.5467963, 'rootMeanSquaredPercentageError': 5271.571, 'rootMeanSquaredLogError': 0.9370826}, 'createTime': '2025-03-25T03:37:41.040899Z', 'modelExplanation': {'meanAttributions': [{'featureAttributions': {'date': 1128.971926141449, 'sale_dollars': 4348.434860813539, 'city': 478.3553377565088, 'county': 165.45809104387462, 'zip_code': 368.8038260627658}}]}}


## Send a batch prediction request

Send a batch prediction to your deployed model.

### Make the batch prediction request

Now that your Model resource is trained, you can make a batch prediction by invoking the `batch_predict()` method using a BigQuery source and destination, with the following parameters:

- `job_display_name`: The human readable name for the batch prediction job.
- `bigquery_source`: BigQuery URI to a table, up to 2000 characters long. For example: `bq://projectId.bqDatasetId.bqTableId`
- `bigquery_destination_prefix`: The BigQuery dataset or table for storing the batch prediction results.
- `instances_format`: The format for the input instances. Since a BigQuery source is used here, this should be set to `bigquery`.
- `predictions_format`: The format for the output predictions, `bigquery` is used here to output to a BigQuery table.
- `generate_explanations`: Set to `True` to generate explanations.
- `sync`: Set **True** to wait until the completion of the job.

In [13]:
batch_predict_bq_output_dataset_name = "iowa_liquor_sales_predictions"
batch_predict_bq_output_dataset_path = "{}.{}".format(
    PROJECT_ID, batch_predict_bq_output_dataset_name
)
batch_predict_bq_output_uri_prefix = "bq://{}.{}".format(
    PROJECT_ID, batch_predict_bq_output_dataset_name
)
# Must be the same location as batch_predict_bq_input_uri
client = bigquery.Client(project=PROJECT_ID)
bq_dataset_id = bigquery.Dataset(batch_predict_bq_output_dataset_path)
dataset_location = "US"  # @param {type : "string"}
bq_dataset_id.location = dataset_location
# delete any existing dataset
try:
    client.delete_dataset(bq_dataset_id, delete_contents=True)
except Exception as e:
    print(e)
bq_dataset = client.create_dataset(bq_dataset_id)
print(
    "Created bigquery dataset {} in {}".format(
        batch_predict_bq_output_dataset_path, dataset_location
    )
)

404 DELETE https://bigquery.googleapis.com/bigquery/v2/projects/asn3-454801/datasets/iowa_liquor_sales_predictions?deleteContents=true&prettyPrint=false: Not found: Dataset asn3-454801:iowa_liquor_sales_predictions
Created bigquery dataset asn3-454801.iowa_liquor_sales_predictions in US


For AutoML models, manual scaling can be adjusted by setting both min and max nodes i.e., `starting_replica_count` and `max_replica_count` as the same value(in this example, set to 1). The node count can be increased or decreased as required by the load

The `batch_predict` method can export predictions either to BigQuery or GCS. In this example, the predictions are exported to BigQuery

In [14]:
PREDICTION_DATASET_BQ_PATH = (
    "bq://bigquery-public-data:iowa_liquor_sales_forecasting.2021_sales_predict"
)

batch_prediction_job = model.batch_predict(
    job_display_name="iowa_liquor_sales_forecasting_predictions",
    bigquery_source=PREDICTION_DATASET_BQ_PATH,
    instances_format="bigquery",
    bigquery_destination_prefix=batch_predict_bq_output_uri_prefix,
    predictions_format="bigquery",
    generate_explanation=True,
    sync=False,
)

print(batch_prediction_job)

<google.cloud.aiplatform.jobs.BatchPredictionJob object at 0x7e4403d925d0> is waiting for upstream dependencies to complete.


### Wait for completion of batch prediction job

Next, wait for the batch job to complete. Alternatively, you can set the `sync` parameter to `True` in the `batch_predict()` method to wait until the batch prediction job is completed.

In [15]:
batch_prediction_job.wait()

### Get the predictions and explanations

Next, get the results from the completed batch prediction job and print them out. Each result row includes the prediction and explanation.

In [16]:
for row in batch_prediction_job.iter_outputs():
    print(row)

Row(('ADAIR', 'ADAIR', '2021-04-02', {'attributions': [{'featureAttributions': {'date': -75.81591796875, 'sale_dollars': -1064.9026692708333, 'city': 76.23038736979167, 'zip_code': -39.8702392578125, 'county': 19.311319986979168}, 'outputDisplayName': None}]}, {'value': 2069.677490234375}, None, "CASEY'S GENERAL STORE #2521 / ADAIR", '50002.0'), {'city': 0, 'county': 1, 'date': 2, 'explanation': 3, 'predicted_sale_dollars': 4, 'sale_dollars': 5, 'store_name': 6, 'zip_code': 7})
Row(('ADAIR', 'ADAIR', '2021-04-30', {'attributions': [{'featureAttributions': {'date': -64.22096761067708, 'sale_dollars': -1084.843282063802, 'city': 68.56606038411458, 'zip_code': -47.75640869140625, 'county': 15.860677083333334}, 'outputDisplayName': None}]}, {'value': 2020.4320068359375}, None, "CASEY'S GENERAL STORE #2521 / ADAIR", '50002.0'), {'city': 0, 'county': 1, 'date': 2, 'explanation': 3, 'predicted_sale_dollars': 4, 'sale_dollars': 5, 'store_name': 6, 'zip_code': 7})
Row(('ADAIR', 'ADAIR', '2021-0

### Visualize the forecasts

Lastly, follow the given link to visualize the generated forecasts in [Data Studio](https://support.google.com/datastudio/answer/6283323?hl=en).
The code block included in this section dynamically generates a Data Studio link that specifies the template, the location of the forecasts, and the query to generate the chart. The data is populated from the forecasts generated using BigQuery options where the destination dataset is `batch_predict_bq_output_dataset_path`.

You can inspect the used template at https://datastudio.google.com/c/u/0/reporting/067f70d2-8cd6-4a4c-a099-292acd1053e8. This was created by Google specifically to view forecasting predictions.

**Note:** The Data Studio dashboard can only show the charts properly when the `batch_predict` job is run successfully using the BigQuery options.

In [17]:
tables = client.list_tables(batch_predict_bq_output_dataset_path)

prediction_table_id = ""
for table in tables:
    if (
        table.table_id.startswith("predictions_")
        and table.table_id > prediction_table_id
    ):
        prediction_table_id = table.table_id
batch_predict_bq_output_uri = "{}.{}".format(
    batch_predict_bq_output_dataset_path, prediction_table_id
)


def _sanitize_bq_uri(bq_uri):
    if bq_uri.startswith("bq://"):
        bq_uri = bq_uri[5:]
    return bq_uri.replace(":", ".")


def get_data_studio_link(
    batch_prediction_bq_input_uri,
    batch_prediction_bq_output_uri,
    time_column,
    time_series_identifier_column,
    target_column,
):
    batch_prediction_bq_input_uri = _sanitize_bq_uri(batch_prediction_bq_input_uri)
    batch_prediction_bq_output_uri = _sanitize_bq_uri(batch_prediction_bq_output_uri)
    base_url = "https://datastudio.google.com/c/u/0/reporting"
    query = (
        "SELECT \\n"
        " CAST(input.{} as DATETIME) timestamp_col,\\n"
        " CAST(input.{} as STRING) time_series_identifier_col,\\n"
        " CAST(input.{} as NUMERIC) historical_values,\\n"
        " CAST(predicted_{}.value as NUMERIC) predicted_values,\\n"
        " * \\n"
        "FROM `{}` input\\n"
        "LEFT JOIN `{}` output\\n"
        "ON\\n"
        "CAST(input.{} as DATETIME) = CAST(output.{} as DATETIME)\\n"
        "AND CAST(input.{} as STRING) = CAST(output.{} as STRING)"
    )
    query = query.format(
        time_column,
        time_series_identifier_column,
        target_column,
        target_column,
        batch_prediction_bq_input_uri,
        batch_prediction_bq_output_uri,
        time_column,
        time_column,
        time_series_identifier_column,
        time_series_identifier_column,
    )
    params = {
        "templateId": "067f70d2-8cd6-4a4c-a099-292acd1053e8",
        "ds0.connector": "BIG_QUERY",
        "ds0.projectId": PROJECT_ID,
        "ds0.billingProjectId": PROJECT_ID,
        "ds0.type": "CUSTOM_QUERY",
        "ds0.sql": query,
    }
    params_str_parts = []
    for k, v in params.items():
        params_str_parts.append('"{}":"{}"'.format(k, v))
    params_str = "".join(["{", ",".join(params_str_parts), "}"])
    return "{}?{}".format(base_url, urllib.parse.urlencode({"params": params_str}))


print(
    get_data_studio_link(
        PREDICTION_DATASET_BQ_PATH,
        batch_predict_bq_output_uri,
        time_column,
        time_series_identifier_column,
        target_column,
    )
)

https://datastudio.google.com/c/u/0/reporting?params=%7B%22templateId%22%3A%22067f70d2-8cd6-4a4c-a099-292acd1053e8%22%2C%22ds0.connector%22%3A%22BIG_QUERY%22%2C%22ds0.projectId%22%3A%22asn3-454801%22%2C%22ds0.billingProjectId%22%3A%22asn3-454801%22%2C%22ds0.type%22%3A%22CUSTOM_QUERY%22%2C%22ds0.sql%22%3A%22SELECT+%5Cn+CAST%28input.date+as+DATETIME%29+timestamp_col%2C%5Cn+CAST%28input.store_name+as+STRING%29+time_series_identifier_col%2C%5Cn+CAST%28input.sale_dollars+as+NUMERIC%29+historical_values%2C%5Cn+CAST%28predicted_sale_dollars.value+as+NUMERIC%29+predicted_values%2C%5Cn+%2A+%5CnFROM+%60bigquery-public-data.iowa_liquor_sales_forecasting.2021_sales_predict%60+input%5CnLEFT+JOIN+%60asn3-454801.iowa_liquor_sales_predictions.predictions_2025_03_24T20_42_29_818Z_315%60+output%5CnON%5CnCAST%28input.date+as+DATETIME%29+%3D+CAST%28output.date+as+DATETIME%29%5CnAND+CAST%28input.store_name+as+STRING%29+%3D+CAST%28output.store_name+as+STRING%29%22%7D


# Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

- Dataset
- AutoML Training Job
- Model
- Batch Prediction Job
- Cloud Storage Bucket

In [ ]:
# Delete dataset
dataset.delete()

# Training job
training_job.delete()

# Delete model
model.delete()

# Delete batch prediction job
batch_prediction_job.delete()

# Delete the dataset
try:
    client.delete_dataset(bq_dataset_id, delete_contents=True, not_found_ok=True)
except Exception as e:
    print(e)

# Set this to true only if you'd like to delete your bucket
delete_bucket = False  # set True for deletion

if delete_bucket:
    ! gsutil rm -r $BUCKET_URI